importar librerias

In [1]:
import pandas as pd
import datetime
import numpy as np

importar CSV

In [2]:
Bookings = pd.read_csv(f'C:/Users/Usuario/Documents/Jupyter/Proyecto/Hotel_Booking.csv', na_values=r'\N')

Bookings.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

Quito las columnas que de antemano ya se que no me van a servir

In [3]:
Bookings = Bookings.drop(['adults', 'children', 'babies', 'meal', 'country', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 
                          'previous_bookings_not_canceled', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 
                          'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests'], axis=1)

Bookings.nunique()

hotel                          2
is_canceled                    2
lead_time                    479
arrival_date_year              3
arrival_date_month            12
arrival_date_week_number      53
arrival_date_day_of_month     31
stays_in_weekend_nights       17
stays_in_week_nights          35
market_segment                 8
reserved_room_type            10
assigned_room_type            12
reservation_status             3
reservation_status_date      926
dtype: int64

Hay dos hoteles. Hay que quedarse con el que tenga más registros.

In [4]:
for i in Bookings['hotel'].unique():
    print(str(i))

Resort Hotel
City Hotel


In [5]:
RH = Bookings.loc[(Bookings['hotel'] == 'Resort Hotel'),:]
CH = Bookings.loc[(Bookings['hotel'] == 'City Hotel'),:]

print('Reservaciones efectivas de Resort Hotel:', RH.shape[0], '\nReservaciones efectivas de City Hotel:', CH.shape[0])

Reservaciones efectivas de Resort Hotel: 40060 
Reservaciones efectivas de City Hotel: 79330


Necesito saber si *reservation_status* y *is_canceled* muestran la misma información

In [6]:
for i in Bookings['is_canceled'].unique():
    print(str(i))

0
1


In [7]:
for i in Bookings['reservation_status'].unique():
    print(str(i))

Check-Out
Canceled
No-Show


In [8]:
CH_Effective = Bookings.loc[(Bookings['hotel'] == 'City Hotel') & (Bookings['is_canceled'] == 0), :]
CH_Canceled = Bookings.loc[(Bookings['hotel'] == 'City Hotel') & (Bookings['is_canceled'] == 1), :]

CHTP = CH_Effective.loc[(CH_Effective['reservation_status'] == 'Check-Out'), :].shape[0]
CHFP = CH_Effective.loc[(CH_Effective['reservation_status'] != 'Check-Out'), :].shape[0]

CHFN = CH_Canceled.loc[(CH_Canceled['reservation_status'] == 'Check-Out'), :].shape[0]
CHTN = CH_Canceled.loc[(CH_Canceled['reservation_status'] != 'Check-Out'), :].shape[0]

print('De las reservaciones del City Hotel, tenemos los siguientes resultados\n', 
      'TP: ', CHTP, '\t', 'FP: ',CHFP, '\n', 'FN: ',CHFN, '\t', 'TN: ',CHTN)

De las reservaciones del City Hotel, tenemos los siguientes resultados
 TP:  46228 	 FP:  0 
 FN:  0 	 TN:  33102


Con esto ya se que los atributos *reservation_status* y *is_canceled* me dicen lo mismo, pero *reservation_status* adicionalmente me dice si la cancelación fue anticipada o fue un *No-Show*, así que me quedo con *reservation_status* y tiro *is_canceled*

In [9]:
CH = CH.drop(['hotel', 'is_canceled'], axis=1)

Reviso nuevamente las características de los atributos en el dataset:

In [10]:
CH.nunique()

lead_time                    453
arrival_date_year              3
arrival_date_month            12
arrival_date_week_number      53
arrival_date_day_of_month     31
stays_in_weekend_nights       14
stays_in_week_nights          29
market_segment                 8
reserved_room_type             8
assigned_room_type             9
reservation_status             3
reservation_status_date      864
dtype: int64

In [11]:
CH.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79330 entries, 40060 to 119389
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   lead_time                  79330 non-null  int64 
 1   arrival_date_year          79330 non-null  int64 
 2   arrival_date_month         79330 non-null  object
 3   arrival_date_week_number   79330 non-null  int64 
 4   arrival_date_day_of_month  79330 non-null  int64 
 5   stays_in_weekend_nights    79330 non-null  int64 
 6   stays_in_week_nights       79330 non-null  int64 
 7   market_segment             79330 non-null  object
 8   reserved_room_type         79330 non-null  object
 9   assigned_room_type         79330 non-null  object
 10  reservation_status         79330 non-null  object
 11  reservation_status_date    79330 non-null  object
dtypes: int64(6), object(6)
memory usage: 7.9+ MB


El .csv proviene de una bitácora de registros, en la que cada entrada representa una reservación. Esto lo vamos a pasar a series de tiempo, en la que cada registro va a representar una TS que corresponde a un día en particular, mientras que las columnas dirán la cantidad de habitaciones reservadas que se tenían para ese día. A ese parámetro le llamaremos *lead*. Entonces, necesito saber el *lead_time* más grande (es decir, cual fue la reservación con mayor anticipación registrada en la bitácora). Ese valor me va a definir la cantidad de columnas que tendrá el nuevo dataset.

In [12]:
CH['lead_time'].max()

np.int64(629)

Ahora, como en el nuevo dataset cada registro representa la TS de las reservaciones para un día en particular, tendremos tantas filas a como días de llegada se tengan en la bitácora. Entonces sacamos la diferencia de días que hay en la bitácora respecto a los días de llegada:

In [13]:
YM = CH['arrival_date_year'].max()
Ym = CH['arrival_date_year'].min()

print('Los registros van del año', Ym, 'al año', YM)

Los registros van del año 2015 al año 2017


Tendremos que formatear una columna de fecha, la cual servirá de etiqueta para las TS. Entonces verificamos no tener fechas erróneas (considerando años bisiestos), y creamos la columna *arrival_date*:

In [14]:
list(CH['arrival_date_month'].unique())

['July',
 'August',
 'September',
 'October',
 'November',
 'December',
 'January',
 'February',
 'March',
 'April',
 'May',
 'June']

In [15]:
Meses = {  'January' : '01',
           'February' : '02',
           'March' : '03',
           'April' : '04',
           'May' : '05',
           'June' : '06',
           'July' : '07',
           'August' : '08',
           'September' : '09',
           'October' : '10',
           'November' : '11',
           'December' : '12'}

CH['arrival_date_month'] = CH['arrival_date_month'].map(Meses).fillna(CH['arrival_date_month'])
list(CH['arrival_date_month'].unique())


['07', '08', '09', '10', '11', '12', '01', '02', '03', '04', '05', '06']

In [16]:
DM = CH['arrival_date_day_of_month'].max()
Dm = CH['arrival_date_day_of_month'].min()

print('Los registros van del día', Dm, 'al día', DM)

Los registros van del día 1 al día 31


In [17]:
for month in sorted(CH['arrival_date_month'].unique()):
    x = CH[CH['arrival_date_month'] == month]['arrival_date_day_of_month'].max()
    print('mes:', month, '\tdía máximo:', x)

mes: 01 	día máximo: 31
mes: 02 	día máximo: 29
mes: 03 	día máximo: 31
mes: 04 	día máximo: 30
mes: 05 	día máximo: 31
mes: 06 	día máximo: 30
mes: 07 	día máximo: 31
mes: 08 	día máximo: 31
mes: 09 	día máximo: 30
mes: 10 	día máximo: 31
mes: 11 	día máximo: 30
mes: 12 	día máximo: 31


In [18]:
for year in sorted(CH['arrival_date_year'].unique()):
    x = CH[(CH['arrival_date_month'] == '02') & (CH['arrival_date_year'] == year)]['arrival_date_day_of_month'].max()
    print('año:', year, '\tmes:', '02', '\tdía máximo:', x)

año: 2015 	mes: 02 	día máximo: nan
año: 2016 	mes: 02 	día máximo: 29
año: 2017 	mes: 02 	día máximo: 28


El 2016 fue bisiesto, así que está bien. Procedemos a crear la columna *arrival_date*:

In [19]:
CH['arrival_date'] = pd.to_datetime(CH['arrival_date_year'].map(str) + '-' + CH['arrival_date_month'].map(str) + '-' + CH['arrival_date_day_of_month'].map(str))

CH = CH.drop(['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month', 'arrival_date_week_number'], axis=1)

DateM = CH['arrival_date'].max()
Datem = CH['arrival_date'].min()

print('Los registros van desde el día', Datem, 'hasta el día', DateM)

Los registros van desde el día 2015-07-01 00:00:00 hasta el día 2017-08-31 00:00:00


¿Qué pasa con la cantidad de días reservados, y con la fecha en la que se cancelan las reservaciones? Para llegar a eso, antes hay que verificar varias condiciones que aseguren que los registros son coherentes. Los siguientes tres puntos serán nuestra hipótesis sobre el comportamiento esperado de las reservaciones:
 - Cuando la reservación es *Canceled*, *reservation_status_date* debe ser menor o igual a *arrival_date*
 - Cuando la reservación es *No-Show*,  *reservation_status_date* debe ser igual a *arrival_date*
 - Cuando la reserva es efectiva, la mayor parte de los casos debería cumplirse que *stays_in_week_nights + stays_in_weekend_nights* sea igual a *reservation_status_date - arrival_date*, sin embargo, cabe la posibilidad de que el huésped se retire antes de la fecha de salida de su reservación, o que extienda la reservación por más días. De cualquier forma, lo que si debe pasar es que *reservation_status_date* sea mayor o igual que *arrival_date*

Para lo anterior vamos a crear una columna temporal llamada *validation*, la cual será un atributo booleano que marcará 0 cuando se cumpla lo que esperamos, y 1 cuando pase lo contrario. También será necesario formatear los demás atributos de fechas a *datetime*, para poder hacer cálculos de forma adecuada:

In [20]:
CH.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79330 entries, 40060 to 119389
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   lead_time                79330 non-null  int64         
 1   stays_in_weekend_nights  79330 non-null  int64         
 2   stays_in_week_nights     79330 non-null  int64         
 3   market_segment           79330 non-null  object        
 4   reserved_room_type       79330 non-null  object        
 5   assigned_room_type       79330 non-null  object        
 6   reservation_status       79330 non-null  object        
 7   reservation_status_date  79330 non-null  object        
 8   arrival_date             79330 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 6.1+ MB


In [21]:
CH['reservation_status_date'].head(10)

40060    03/07/2015
40061    01/07/2015
40062    30/04/2015
40063    23/06/2015
40064    02/04/2015
40065    25/06/2015
40066    05/07/2015
40067    25/06/2015
40068    02/07/2015
40069    02/07/2015
Name: reservation_status_date, dtype: object

In [22]:
CH['reservation_status_date'] = pd.to_datetime(CH['reservation_status_date'].str[6:11] + '-' + CH['reservation_status_date'].str[3:5] + '-' + CH['reservation_status_date'].str[0:2])
CH['reservation_status_date'].head(10)

40060   2015-07-03
40061   2015-07-01
40062   2015-04-30
40063   2015-06-23
40064   2015-04-02
40065   2015-06-25
40066   2015-07-05
40067   2015-06-25
40068   2015-07-02
40069   2015-07-02
Name: reservation_status_date, dtype: datetime64[ns]

Creamos un atributo de *effective_days*, el cual guardará la cantidad de días en los que la reservación fue efectiva, y utilizamos el atributo *validation* como auxiliar verificar que las tres hipótesis que se expusieron anteriormente se cumplen:

In [23]:
CH['effective_days'] = (CH['reservation_status_date'] - CH['arrival_date']).dt.days

CH['validation'] = [1 if x > 0 else 0 for x in CH['effective_days']] 
CH[CH['reservation_status'] == 'Canceled']['validation'].unique()


array([0])

In [24]:
CH['validation'] = [1 if x != 0 else 0 for x in CH['effective_days']] 
CH[CH['reservation_status'] == 'No-Show']['validation'].unique()


array([0])

In [25]:
CH['validation'] = [1 if x < 0 else 0 for x in CH['effective_days']] 
CH[CH['reservation_status'] == 'Check-Out']['validation'].unique()


array([0])

Como en los tres casos tenemos que *validation* solo toma valores iguales a cero, ya confirmamos las tres hipótesis. Ahora bien, respecto a los últimos registros en el dataset, estos corresponden a reservaciones para las últimas series de tiempo. 

In [26]:
CH.tail()

,lead_time,stays_in_weekend_nights,stays_in_week_nights,market_segment,reserved_room_type,assigned_room_type,reservation_status,reservation_status_date,arrival_date,effective_days,validation
119385,23,2,5,Offline TA/TO,A,A,Check-Out,2017-09-06,2017-08-30,7,0
119386,102,2,5,Online TA,E,E,Check-Out,2017-09-07,2017-08-31,7,0
119387,34,2,5,Online TA,D,D,Check-Out,2017-09-07,2017-08-31,7,0
119388,109,2,5,Online TA,A,A,Check-Out,2017-09-07,2017-08-31,7,0
119389,205,2,7,Online TA,A,A,Check-Out,2017-09-07,2017-08-29,9,0


Por ejemplo, para el registro con *id = 79329*, la reservación se hace con llegada el *2017-08-29* y reserva nueve días. Cuando hagamos la transformación de los datos y cambiemos de espacio, la última serie de tiempo corresponderá al *2017-08-31*, pero esta reservación llega hasta el *2017-09-06*. Las reservaciones que son después del *2017-08-31* ya no me interesan porque serán series de tiempo incompletas (ya que no podré ver cómo cierra la ocupación). Entonces, hay que truncar estas reservaciones para que, al momento de cambiar de dataset, no tengamos instrucciones que busquen guardar información en fechas posteriores a nuestra última TS. Para ello creamos dos atributos adicionales: *days_reserved* y *leaving_date*, los cuales servirán de referencia junto a *reservation_status* para adaptar los valores de *reservation_status_date* y *effective_days*:

In [27]:
CH.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79330 entries, 40060 to 119389
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   lead_time                79330 non-null  int64         
 1   stays_in_weekend_nights  79330 non-null  int64         
 2   stays_in_week_nights     79330 non-null  int64         
 3   market_segment           79330 non-null  object        
 4   reserved_room_type       79330 non-null  object        
 5   assigned_room_type       79330 non-null  object        
 6   reservation_status       79330 non-null  object        
 7   reservation_status_date  79330 non-null  datetime64[ns]
 8   arrival_date             79330 non-null  datetime64[ns]
 9   effective_days           79330 non-null  int64         
 10  validation               79330 non-null  int64         
dtypes: datetime64[ns](2), int64(5), object(4)
memory usage: 7.3+ MB


In [28]:
CH = CH.reset_index(drop = True)

CH['days_reserved'] = (CH['stays_in_week_nights'] + CH['stays_in_weekend_nights']).astype(int)

CH['leaving_date'] = 0
for i in range(0, len(CH)):
    CH.iloc[i, 12] = CH.iloc[i, 8] + datetime.timedelta(days = int(CH.iloc[i, 11]))
    if CH.iloc[i, 12] > DateM:
        CH.iloc[i, 11] = (DateM - CH.iloc[i, 8]).days + 1
        if CH.iloc[i, 6] == 'Check-Out':
            CH.iloc[i, 7] = DateM + datetime.timedelta(days = 1)
            CH.iloc[i, 9] = (CH.iloc[i, 7] - CH.iloc[i, 8]).days

CH = CH.drop(['leaving_date'], axis=1)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20348\1416503358.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2015-07-03 00:00:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  CH.iloc[i, 12] = CH.iloc[i, 8] + datetime.timedelta(days = int(CH.iloc[i, 11]))


También hay que considerar que existen clientes que salen anticipadamente, o extienden su estancia. Estos están bajo el estatus de *Check-Out*, pero su estancia no tiene la misma duración que lo que se indica en la reservación:

In [29]:
CH['validation'] = (CH['days_reserved'] == CH['effective_days'])
CH[(CH['validation'] == 0) & (CH['reservation_status'] == 'Check-Out')]['validation'].count()


np.int64(6)

In [30]:
CH[(CH['validation'] == 0) & (CH['reservation_status'] == 'Check-Out')][['days_reserved', 'effective_days']]

,days_reserved,effective_days
234,4,0
6121,9,1
30600,2,0
73461,1,0
73942,1,0
73943,1,0


Son solo seis registros, y estos hicieron el *Check-Out* antes de la fecha de salida de la reservación, lo cual es bastante normal considerando que tenemos registros de 2 años completos, así que, para los casos en que el huésped se retira antes de terminar con el periodo de su estancia, cancelaremos el resto de noches reservadas el día en que haga su *Check-Out*. Para el resto de reservaciones con estatus de *Canceled* y *No-Show* se cancelarán el día que indica el atributo *reservation_status_date*.

In [31]:
CH = CH.drop(['stays_in_week_nights', 'stays_in_weekend_nights'], axis=1)

CH.loc[(CH['validation'] == 0) & (CH['reservation_status'] == 'Check-Out') & (CH['effective_days'] == 0), 'effective_days'] = 1

CH[(CH['validation'] == 0) & (CH['reservation_status'] == 'Check-Out')][['days_reserved', 'effective_days']]

,days_reserved,effective_days
234,4,1
6121,9,1
30600,2,1
73461,1,1
73942,1,1
73943,1,1


Respecto a los atributos *reserved_room_type* y *assigned_room_type* realmente lo único relevante es la habitación reservada, ya que esa es la que contabiliza y afecta el inventario disponible. La asignación se realiza durante el *Check-In* y los motivos por los que puede cambiar el tipo de habitación es porque el cliente decidió incrementar la categoría, porque hubo sobreventa o porque la habitación reservada se encuentra en algún tipo de mantenimiento. Podría darse el caso en el que el huésped cancele su reservación debido a la disponibilidad de los tipos de habitación, pero como no tenemos forma de saber esto (si es que sucede), entonces daremos por hecho que *assigned_room_type* no afecta absolutamente en nada al comportamiento de las habitaciones reservadas.

In [32]:
CH.loc[(CH['reservation_status'] == 'Canceled') & (CH['effective_days'] < 0), 'effective_days'] = 0
CH.loc[(CH['reservation_status'] == 'Canceled') & (CH['effective_days'] < 0), 'effective_days'].count()


np.int64(0)

In [33]:
CH = CH.drop(['assigned_room_type', 'validation'], axis=1)
CH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79330 entries, 0 to 79329
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   lead_time                79330 non-null  int64         
 1   market_segment           79330 non-null  object        
 2   reserved_room_type       79330 non-null  object        
 3   reservation_status       79330 non-null  object        
 4   reservation_status_date  79330 non-null  datetime64[ns]
 5   arrival_date             79330 non-null  datetime64[ns]
 6   effective_days           79330 non-null  int64         
 7   days_reserved            79330 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 4.8+ MB


Para hacer la transformación al nuevo dataset de series de tiempo, necesito saber las dimensiones que tendrá. Además se tiene la intención de descomponer la TS principal de un día en varias TS, en donde cada una represente las reservaciones realizadas bajo ciertas condiciones. Un ejemplo de esto podría ser el descomponer las TS principales por segmentos de mercado y por tipos de habitación, pudiendo crear un modelo específico para cada componente:

In [34]:
m = CH['lead_time'].max() + CH['days_reserved'].max()
n = (CH['arrival_date'].max() - CH['arrival_date'].min()).days
ms = CH['market_segment'].nunique()
rt = CH['reserved_room_type'].nunique()

print('Dimension del dataset: [', n, 'x', m, ']\nTipos de habitaciones:', rt, '\nSegmentos de mercado:', ms, '\nTotal de posibles combinaciones:', ms*rt)

Dimension del dataset: [ 792 x 686 ]
Tipos de habitaciones: 8 
Segmentos de mercado: 8 
Total de posibles combinaciones: 64


Como aún no tengo una TS como para analizar cual tratamiento me conviene más, buscaremos ir desde lo particular hacia lo general. Esto es, vamos a descomponer la TS ya que siempre podremos recuperar la TS principal simplemente sumándolas, pero no podemos hacer la descomposición a partir de la TS principal. Aún así, hacer 64 modelos parece un tanto ambicioso, así que me voy a limitar a usar 8 datasets: uno por cada segmento de mercado. Solo falta definir las reglas para hacer el cambio de espacio de los datos.
Necesito transformar los atributos de fechas para tener nuevos atributos que describan fechas relativas a la fecha de referencia de la TS. Es decir, si una TS corresponde al 01/01/2016 (que vendría a representar el *arrival_date*, entonces quisiera saber con cuantos días de anticipación se hizo la reservación respecto a la fecha de llegada. esto también aplica para las cancelaciones). Para ello crearemos un nuevo atributo, llamado *last_update*, en el cual se cumplirá lo siguiente:
- si *last_update < 0*, implica que *reservation_status = Canceled*, ya que no existe un *No-Show* anticipado ni un *Check-Out* anticipado.
- de forma análoga, si *last_update > 0*, entonces *reservation_status = Check-Out*
- Por último, si *last_update == 0*, entonces *reservation_status* podría tomar cualquier valor, por lo que habría que verificar para saber la implicación de ese registro en las TS: 

In [35]:
CH = CH.drop(['reserved_room_type'], axis = 1)

CH['last_update'] = (CH['arrival_date'] - CH['reservation_status_date']).dt.days


Simplificamos los nombres de los segmentos e inicializamos los nuevos datasets en donde se guardarán las TS ya descompuestas:

In [36]:
list(CH['market_segment'].unique())

['Offline TA/TO',
 'Online TA',
 'Groups',
 'Complementary',
 'Direct',
 'Corporate',
 'Undefined',
 'Aviation']

In [37]:
CH.loc[CH['market_segment'] == 'Offline TA/TO', 'market_segment'] = 'Offline'
CH.loc[CH['market_segment'] == 'Online TA', 'market_segment'] = 'Online'

a = ['date']
b = []
for i in range(0, m + 1):
    b = b + [i]

Offline = pd.DataFrame(0, np.arange(n + 1), a + b)
Online = pd.DataFrame(0, np.arange(n + 1), a + b)
Groups = pd.DataFrame(0, np.arange(n + 1), a + b)
Complementary = pd.DataFrame(0, np.arange(n + 1), a + b)
Direct = pd.DataFrame(0, np.arange(n + 1), a + b)
Corporate = pd.DataFrame(0, np.arange(n + 1), a + b)
Undefined = pd.DataFrame(0, np.arange(n + 1), a + b)
Aviation = pd.DataFrame(0, np.arange(n + 1), a + b)

DataFrames = [Offline, Online, Groups, Complementary, Direct, Corporate, Undefined, Aviation]
segment = {'Offline': 0, 'Online': 1, 'Groups': 2, 'Complementary': 3, 'Direct': 4, 'Corporate': 5, 'Undefined': 6, 'Aviation': 7}


minDate = CH['arrival_date'].min()
for x in DataFrames:
    x['date'] = pd.date_range(minDate, periods = n + 1)

DataFrames[0].head()

,date,0,1,2,3,4,5,6,7,8,...,677,678,679,680,681,682,683,684,685,686
0,2015-07-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-07-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015-07-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2015-07-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015-07-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Creamos una función que va agregar las reservaciones dentro de la serie de tiempo que le corresponda, en donde el parámetro *mode* va a indicar con *+1* si se agrega una reservación y con *-1* si se elimina:

In [38]:
def modify_series(df, arrival, days, lead, mode):
    date_start = (arrival - minDate).days
    date_end = date_start + days 
    k = 1
    for i in range(date_start, date_end):
        df.iloc[i, 1:lead + k + 1] = df.iloc[i, 1:lead + k + 1] + mode
        k = k + 1
        

Y ahora leemos todos los registros que hay en la bitácora para reflejar las reservaciones en los nuevos dataset, así como los movimientos de *Canceled*, *No-Show* y *Check-Out* que corresponden.

In [39]:
CH['days_diff'] = CH['days_reserved'] - CH['effective_days']

CH.to_csv('Cleaned_CH.csv', index = False)

for i in range(0, len(CH)):
    modify_series(DataFrames[segment[CH.iloc[i, 1]]], CH.iloc[i, 4], CH.iloc[i, 6], CH.iloc[i, 0], 1)
    
    match CH.iloc[i, 2]:
        case 'Canceled':
            modify_series(DataFrames[segment[CH.iloc[i, 1]]], CH.iloc[i, 4], CH.iloc[i, 6], CH.iloc[i, 7], -1)
        
        case 'No-Show':
            modify_series(DataFrames[segment[CH.iloc[i, 1]]], CH.iloc[i, 4], CH.iloc[i, 6], 0, -1)
    
        case 'Check-Out': 
            if CH.iloc[i, 8] > 0:
                modify_series(DataFrames[segment[CH.iloc[i, 1]]], CH.iloc[i, 4] + datetime.timedelta(days = int(CH.iloc[i, 5])), CH.iloc[i, 8], 0, -1)
                    
            elif CH.iloc[i, 8] < 0:
                modify_series(DataFrames[segment[CH.iloc[i, 1]]], CH.iloc[i, 4] + datetime.timedelta(days = int(CH.iloc[i, 6])), -1 * CH.iloc[i, 8], 0, 1)

for x in list(CH['market_segment'].unique()):
    DataFrames[segment[x]].to_csv(x + '.csv', index = False) 